# Softmax  regression model

The following notebook describes the creation of a softmax  regression model for the classification of german traffic signs, using tensorflow. 

In [1]:
# Load libraries
from sklearn.metrics import accuracy_score
from PIL import Image
import glob
import inspect
import numpy as np
import os
import pandas as pd
import pickle
import random
import tensorflow as tf


C:\Users\eduardo\Anaconda\Miniconda3\envs\dl\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Location of the train and test images
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
ruta_train = os.sep.join([current_dir,'..','images','train'])
ruta_test = os.sep.join([current_dir,'..','images','test'])


In [3]:
# Load train and test images

train_images = []
test_images = []

y_train = []
y_test = []

for filename in glob.iglob(ruta_train + os.sep + '*.ppm'):
    label = int(filename.split(os.sep)[-1].split('_')[0])
    y_train.append(label)
    train_images.append(Image.open(filename))

for filename in glob.iglob(ruta_test + os.sep + '*.ppm'):
    label = int(filename.split(os.sep)[-1].split('_')[0])
    y_test.append(label)
    test_images.append(Image.open(filename))
    

In [4]:
# Helper functions

# Resizes an image to a defined size, and then reshapes into a 1xn array
def process_img(img,size):
    out = img.resize(size, Image.ANTIALIAS)
    out = np.asarray(out)
    out = out.reshape(1, -1)
    return out

# Applies the process_img function to the train and test sets of images, and return train and test arrays
def formar_set(images,im_size):
    X = [process_img(img, im_size) for img in images]
    X = np.vstack(X)
    return X


In [6]:
# Set parameters
im_size = (35,35)
learning_rate = 0.0002
batch_size = 500
training_iteration = 10000 # max number of iterations
termination_margin = 0.001/100  # If the cost function varies less than this between iterations, the training process stops.


In [7]:
# Create and normalize training and test feature sets

X_train = formar_set(train_images,im_size)
X_test = formar_set(test_images,im_size)
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Convert class vectors (y_train and y_test) to binary class matrices

Y_train = np.array(pd.get_dummies(pd.Series(y_train)))
Y_test = np.array(pd.get_dummies(pd.Series(y_test)))


In [8]:
## Create tf model

size_x = X_train.shape[1]
size_y = Y_train.shape[1]

# TF graph input
x = tf.placeholder("float", [None, size_x], name='x') 
y = tf.placeholder("float", [None, size_y], name='y') 

# Set model weights
W = tf.Variable(tf.zeros([size_x, size_y]), name='W')
b = tf.Variable(tf.zeros([size_y]), name='b')

# Construct a linear model
model = tf.nn.softmax(tf.matmul(x, W) + b, name='model') # Softmax

# Define cross_entropy as the cost function
cost_function = -tf.reduce_sum(y*tf.log(model))
                
# Define Gradient Descent as the model optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

# Initializing the variables
init = tf.global_variables_initializer()

In [9]:
# Launch the graph
with tf.Session() as sess:
    # Initialize variables and calculate initial cost
    sess.run(init)
    initial_cost = sess.run(cost_function,{x:X_test,y:Y_test})
    cond = True
    overfit = False
    cont = 0
  
    # Training cycle
    while cond:
        cont += 1
        rand_ind = random.sample(range(len(X_train)),batch_size)
        batch_xs, batch_ys = X_train[rand_ind,:],Y_train[rand_ind,:]
        
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
        cost_epoch = sess.run(cost_function,{x:X_test,y:Y_test})
        
        # Progress is printed every 200 iterations
        if cont % 200 == 0:
            print("Iteration:", '%04d' % (cont), "cost=", "{:.9f}".format(cost_epoch))
            
        # Progress is checked every 50 iterations
        if cont % 50 == 0:
            overfit = (cost_epoch > initial_cost) and (abs(cost_epoch - initial_cost)/initial_cost < termination_margin)
        
        # Termination condition
        if (cont >= training_iteration) or overfit:
            print("Iteration:", '%04d' % (cont), "cost=", "{:.9f}".format(cost_epoch))
            cond = False
        else:
            initial_cost = cost_epoch
        
    print("Tuning completed!")
    
    # Measuring performance
    sess.run(cost_function,{x:X_test,y:Y_test})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(model,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,'float'))

    print('Training accuracy: {0}'.format(sess.run(accuracy, feed_dict={x: X_train, y: Y_train})))
    print('Test accuracy: {0}'.format(sess.run(accuracy, feed_dict={x: X_test, y: Y_test})))
    
    #Create a saver object which will save all the variables
    model_location = os.sep.join([current_dir,'..','models','model2','model2'])
    saver = tf.train.Saver()
    saver.save(sess, model_location,global_step=1000)
  


Iteration: 0200 cost= 349.781280518
Iteration: 0400 cost= 276.740875244
Iteration: 0600 cost= 246.049758911
Iteration: 0800 cost= 226.489227295
Iteration: 1000 cost= 213.862579346
Iteration: 1200 cost= 204.127548218
Iteration: 1400 cost= 196.714675903
Iteration: 1600 cost= 191.198043823
Iteration: 1800 cost= 188.057296753
Iteration: 2000 cost= 184.019729614
Iteration: 2200 cost= 181.160476685
Iteration: 2400 cost= 179.066223145
Iteration: 2600 cost= 177.047363281
Iteration: 2800 cost= 175.037857056
Iteration: 3000 cost= 173.926071167
Iteration: 3200 cost= 172.691268921
Iteration: 3400 cost= 171.041671753
Iteration: 3600 cost= 169.614288330
Iteration: 3800 cost= 169.597244263
Iteration: 4000 cost= 168.817108154
Iteration: 4200 cost= 167.770507812
Iteration: 4400 cost= 167.882644653
Iteration: 4600 cost= 167.668914795
Iteration: 4800 cost= 167.078643799
Iteration: 5000 cost= 166.804183960
Iteration: 5200 cost= 166.186126709
Iteration: 5400 cost= 166.091720581
Iteration: 5600 cost= 165.81

In [10]:
x_s = X_test[166:180]
# x_s = x_s.reshape(1,-1)

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('C:/Users/eduardo/Desktop/Reto Kiwi/German traffic signs detector/models/model2/model2-1000.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('C:/Users/eduardo/Desktop/Reto Kiwi/German traffic signs detector/models/model2/'))
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    W = graph.get_tensor_by_name("W:0")
    b = graph.get_tensor_by_name("b:0")
    model = graph.get_tensor_by_name("model:0")
    pred = tf.argmax(model,1)
    
    print(sess.run(pred,feed_dict={x:x_s}))

    
print(y_test[166:180])

INFO:tensorflow:Restoring parameters from C:\Users\eduardo\Desktop\Reto Kiwi\German traffic signs detector\reports\..\models\model2\model2-1000
[15 16 16 17 17 17 17 17 17 18 26 18 18 18]
[15, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18]
